In [4]:
import pandas as pd
import xlrd
import os

# Your current directory (including python script & all excel files)
mydir = (os.getcwd()).replace('\\','/') + '/'

#Get all excel files include subdir
filelist=[]
for path, subdirs, files in os.walk(mydir):
    for file in files:
        if (file.endswith('.xlsx') or file.endswith('.xls') or file.endswith('.XLS')):
            filelist.append(file)
print(filelist)

['欠操.xlsx', '機掰.xlsx', '黑鬼.xlsx', '腦破.xlsx', '雞掰.xlsx', '廢物.xlsx', '婊子.xlsx', '畜生.xlsx', '垃圾.xlsx', '恐龍妹.xlsx', '支畜.xlsx', '肥宅.xlsx', '精障.xlsx', '醜男.xlsx', '欠揍.xlsx', '弱智.xlsx', '腦殘.xlsx', '破腦.xlsx', '欠幹.xlsx', '噁男.xlsx', '白痴.xlsx', '幹你.xlsx', '死胖.xlsx', '變態.xlsx', '破麻.xlsx', '神經病.xlsx', '肥豬.xlsx', '低能.xlsx', '智障.xlsx', '欠扁.xlsx', '喜憨兒.xlsx', '白目.xlsx', '遲緩.xlsx', '人妖.xlsx', '娘炮.xlsx', '母豬.xlsx', '無腦.xlsx', '操你.xlsx', '臭甲.xlsx', '啟智.xlsx', '番仔.xlsx', '臭婊.xlsx', '港仔.xlsx', '醜女.xlsx', '老處女.xlsx', '黑奴.xlsx', '屁孩.xlsx', '妓女.xlsx', '怪胎.xlsx']


In [7]:
import re
for file in filelist:
    filename = re.sub(r'.xlsx','', file)
    print(filename)

欠操
機掰
黑鬼
腦破
雞掰
廢物
婊子
畜生
垃圾
恐龍妹
支畜
肥宅
精障
醜男
欠揍
弱智
腦殘
破腦
欠幹
噁男
白痴
幹你
死胖
變態
破麻
神經病
肥豬
低能
智障
欠扁
喜憨兒
白目
遲緩
人妖
娘炮
母豬
無腦
操你
臭甲
啟智
番仔
臭婊
港仔
醜女
老處女
黑奴
屁孩
妓女
怪胎


In [11]:
from opencc import OpenCC
import ast
import re

def filter_emoji(desstr, restr=''):
    try:
        co = re.compile(u'[\U00010000-\U0010ffff]')
    except:
        co = re.compile(u'[\uD800-\uDBFF][\uDC00-\uDFFF]')
    return co.sub(restr, desstr)

cc = OpenCC('s2tw')

for file in filelist:
    df = pd.read_excel(file, usecols=["Post Description","Comments"])
    comments = df["Comments"]
    # print(comments)
    comments_dicts = comments.to_dict()

    posts_list = []
    
    for key, value in comments_dicts.items():
        val_str = comments_dicts[key]
        try:
            val_dict = ast.literal_eval(val_str)
        except SyntaxError:
            minus = -2
            index_list = [pos for pos, char in enumerate(val_str) if char == ',']
            while(True):
                try:
                    new_val_str = val_str[:index_list[minus]] + "}"
                    val_dict = ast.literal_eval(new_val_str)
                    break
                except SyntaxError:
                    minus -=1
        except ValueError:
            val_str = "{}"
        post_list = list(val_dict.values())
        for index, item in enumerate(post_list):
            post_list[index] = filter_emoji(post_list[index])
            post_list[index] = re.sub(r'(http|https)?:\/\/\S*', '', post_list[index], flags=re.MULTILINE)
            post_list[index] = re.sub(r'[0-9\t\n\s+-—•·.。⋯…～~”“？?！!=「」『』（）()《》【】\u200d♀️♂☜☞⚠#♥❤╰⊙╮◉‿❗✊]+', '', post_list[index])
            if post_list[index].encode('utf-8').isalpha():
                post_list[index] = ""
            if len(post_list[index]) < 4:
                post_list[index] = ""
            elif len(post_list[index]) > 30:
                post_list[index] = ""
            post_list[index] = cc.convert(post_list[index])
        post_list = list(filter(None, post_list))
        posts_list.extend(post_list)
    
    filename = re.sub(r'.xlsx','', file)
    
    textfile = open(filename+".txt","w")
    for element in posts_list:
        textfile.write(element + '\n')
    textfile.close()